In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Store CSV into DataFrame

In [2]:
national_csv = "data_sources/national_data.csv"
national_data_df = pd.read_csv(national_csv)
national_data_df.head()

,date,states,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,lastModified,total,posNeg,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease,hash,date_modified
0,20201117,56,11209335,137833090,9281.0,76958.0,524678.0,14851.0,27681.0,4379.0,...,2020-11-17T24:00:00Z,0,0,1555,3915,1092462,156722,1503534,708da364131cc93957705f7e156adc77b09ec479,2020-11-17
1,20201116,56,11052613,136740628,13639.0,73268.0,520763.0,14317.0,27437.0,4166.0,...,2020-11-16T24:00:00Z,0,0,637,3148,1103527,148868,1478180,03274b3fa0fd66a6b690404cfd1a02200de4da12,2020-11-16
2,20201115,56,10903745,135637101,13654.0,70113.0,517615.0,13693.0,27269.0,3939.0,...,2020-11-15T24:00:00Z,0,0,677,1876,1162554,144434,1614984,b54fb425773fd3a742135208376358ed968ced59,2020-11-15
3,20201114,56,10759311,134474547,13657.0,69498.0,515739.0,13337.0,27172.0,3945.0,...,2020-11-14T24:00:00Z,0,0,1312,3465,1174712,163540,1661289,37d336c0c6780c7cb7e642f3b913b33bae511c5b,2020-11-14
4,20201113,56,10595771,133299835,12010.0,68496.0,512274.0,13132.0,26997.0,3769.0,...,2020-11-13T24:00:00Z,0,0,1275,3693,1237330,172106,1695579,a88e3bd0b213dff8125c5337801650cdd9b39c15,2020-11-13


In [3]:
national_data_df.columns

Index(['date', 'states', 'positive', 'negative', 'pending',
       'hospitalizedCurrently', 'hospitalizedCumulative', 'inIcuCurrently',
       'inIcuCumulative', 'onVentilatorCurrently', 'onVentilatorCumulative',
       'recovered', 'dateChecked', 'death', 'hospitalized', 'totalTestResults',
       'lastModified', 'total', 'posNeg', 'deathIncrease',
       'hospitalizedIncrease', 'negativeIncrease', 'positiveIncrease',
       'totalTestResultsIncrease', 'hash', 'date_modified'],
      dtype='object')

# Clean Data

In [4]:
new_national_data_df = national_data_df[['date_modified','positive',
                                    'hospitalizedCumulative','death',
                                    'recovered']].copy()
national_data_transformed = new_national_data_df.rename(columns={
    "Date": "date_modified", "Positive" : "positive",
    "Hospitalized": "hospitalizedCumulative", "Deaths": "death",
    "Recovered": "recovered"})
national_data_transformed.head()


,date_modified,positive,hospitalizedCumulative,death,recovered
0,2020-11-17,11209335,524678.0,239835.0,4294743.0
1,2020-11-16,11052613,520763.0,238280.0,4245304.0
2,2020-11-15,10903745,517615.0,237643.0,4185979.0
3,2020-11-14,10759311,515739.0,236966.0,4149020.0
4,2020-11-13,10595771,512274.0,235654.0,4095749.0


## Connect to local database

In [5]:
rds_connection_string = "postgres:postgres@localhost:5432/COVID-19"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [6]:
engine.table_names()

['national_stats']

## CSV converted DataFrame into database

In [9]:
national_data_transformed.to_sql(name='national_stats', con=engine, if_exists='append', index=False)


## Load DataFrame into database

In [10]:
pd.read_sql_query('select * from national_stats', con=engine).head()


,date_modified,positive,hospitalizedCumulative,death,recovered
0,2020-11-17,11209335,524678.0,239835.0,4294743.0
1,2020-11-16,11052613,520763.0,238280.0,4245304.0
2,2020-11-15,10903745,517615.0,237643.0,4185979.0
3,2020-11-14,10759311,515739.0,236966.0,4149020.0
4,2020-11-13,10595771,512274.0,235654.0,4095749.0
